# ALL FUNCTIONS AND CODE BITS DEFINED DURING COURSE

In [2]:
# Imports
import numpy as np
import scipy
from scipy.stats import norm
import matplotlib.pyplot as plt
import math
import sys
import itertools

from iminuit import Minuit 
from scipy.stats import binom, poisson
from scipy.stats import chi2

sys.path.append('/AppStat/AppStat2019_local/External_Functions')
from ExternalFunctions import Chi2Regression
from ExternalFunctions import nice_string_output, add_text_to_ax

ModuleNotFoundError: No module named 'ExternalFunctions'

### Weighted mean

In [4]:
def weighted_mean(data,weights):
    mean = 0
    weighted_sum = 0
    
    if sum(weights) == 1:
        for i in range(len(data)):
            mean = mean + data[i]*weights[i]
        mean = mean/len(weights)
    
    else:
        for i in range(len(data)):
            mean = mean + data[i]*weights[i]
            weighted_sum = weighted_sum + weights[i]
        mean = mean/weighted_sum

    return mean

### Weighted STD

In [5]:
def weighted_std(data, weights, mean):
    std = 0
    weighted_sum = 0
    
    for i in range(len(data)):
        std = std + weights[i]*(data[i]-mean)**2
        weighted_sum = weighted_sum + weights[i]
    std = std/(weighted_sum)
    std = np.sqrt(std)

    return std

### Normalized Gaussian

In [3]:
def func_gaussian_norm(x, sigma, mu, N) :
    result = N/(np.sqrt(2*np.pi))*np.exp(-0.5*((x-mu)/sigma)**2)
    
    return result

### Gaussian

In [6]:
def func_gaussian(x, sigma, mu):
    result = 1/(sigma*np.sqrt(2*np.pi))*np.exp(-0.5*((x-mu)/sigma)**2)
    
    return result

### CDF - Gaussian integration

In [7]:
def gaus_between_a_b(a,b):
    p_value = 2*(norm.cdf(a)-norm.cdf(b))
    return p_value

### Binomial

In [1]:
def binom_func(n,p,N):
    P = math.factorial(n)/(math.factorial(N)*math.factorial(n-N)) * p**N * (1-p)**(n-N)
    return P

### Binomial - probability of getting P and over

In [2]:
def prob_of_above_P(P,n,p):
    prob_to_P = 0
    for N_sum in range(0,P):                              #If you don't play you can't win
        prob_to_P = prob_to_P + binom_func(n,p,N_sum)

    prob_P_plus_1_and_over = 1-prob_to_P
    return prob_P_plus_1_and_over

### Poissonian

In [3]:
def Poissonian_func(k,Lambda):
    P_pois = Lambda**k * np.exp(-Lambda)/(math.factorial(k))
    return P_pois

### Calculation of Chi2

In [4]:
def chi2_calculations(observed, expected, errors):
    chi2_calc = 0
    
    if type(expected) != list or type(expected) != np.array: 
        expected = expected*np.ones(len(observed))
    if type(errors) != list or type(errors) != np.array: 
        errors = errors*np.ones(len(observed))
    
    chi2_list = []
    for i in range (0,len(observed)):
        if errors[i] != 0:
            chi2_calc = chi2_calc + (observed[i]-expected[i])**2/errors[i]**2
            
            chi2_list.append((observed[i]-expected[i])**2/errors[i]**2)
    return chi2_calc, chi2_list

### Multiple (here 3) Gaussians on slope

In [8]:
def func_gpol0(x,a,mu1,sigma1,b,mu2,sigma2,c,mu3,sigma3, cst,slope) :
    norm1 = binwidth_gauss * a / np.sqrt(2.0*np.pi) / sigma1
    norm2 = binwidth_gauss * b / np.sqrt(2.0*np.pi) / sigma2
    norm3 = binwidth_gauss * c / np.sqrt(2.0*np.pi) / sigma3
    z1 = (x-mu1)/sigma1
    z2 = (x-mu2)/sigma2
    z3 = (x-mu3)/sigma3
   
    return norm1 * np.exp(-0.5*z1*z1) + cst + norm2 * np.exp(-0.5*z2*z2) + norm3 * np.exp(-0.5*z3*z3) + slope*x

### Correlation of 2 parameters

In [5]:
#Error propagation
def dfdx(param1,param2,param3):
    
    derivative = param1 + param2 + param3 #Find your derivative!!!
    
    return derivative

def dfdy(param1,param2,param3):
    
    derivative = param1 + param2 + param3 #Find your derivative!!!
    
    return derivative

def sig_tot_corr(dfdx,x_err,dfdy,y_err,rho):
    V = x_err*y_err*rho
    Sig_tot_corr = np.sqrt((dfdx(param1,param2,param3)**2)*x_err**2 + (dfdy(param1,param2,param3)**2)*y_err**2 + 
                          2*dfdx(param1,param2,param3)*(dfdy(param1,param2,param3))*V)
    return Sig_tot_corr

### Covariance and correlation coefficient

In [12]:
def cov_corr(A,B):
    meanA = np.mean(A)
    meanB = np.mean(B)
    stdA = np.std(A)
    stdB = np.std(B)
    Covariance = np.mean((A-meanA)*(B-meanB))
    CorrCoeff = Covariance/(stdA*stdB)
    return Covariance, CorrCoeff

### ROC-CURVE

In [1]:
# Calculate ROC curve from two histograms (hist1 is signal, hist2 is background):
def calc_ROC(hist1, hist2) :

    # First we extract the entries (y values) and the edges of the histograms
    y_sig, x_sig_edges, _ = hist1 
    y_bkg, x_bkg_edges, _ = hist2
    
    # Check that the two histograms have the same x edges:
    if np.array_equal(x_sig_edges, x_bkg_edges) :
        
        # Extract the center positions (x values) of the bins (both signal or background works - equal binning)
        x_centers = 0.5*(x_sig_edges[1:] + x_sig_edges[:-1])
        
        # Calculate the integral (sum) of the signal and background:
        integral_sig = y_sig.sum()
        integral_bkg = y_bkg.sum()
    
        # Initialize empty arrays for the True Positive Rate (TPR) and the False Positive Rate (FPR):
        TPR = np.zeros_like(y_sig) # True positive rate (sensitivity)
        FPR = np.zeros_like(y_sig) # False positive rate ()
        
        # Loop over all bins (x_centers) of the histograms and calculate TN, FP, FN, TP, FPR, and TPR for each bin:
        for i, x in enumerate(x_centers): 
            
            # The cut mask
            cut = (x_centers < x)
            
            # True positive
            TP = np.sum(y_sig[~cut]) / integral_sig    # True positives
            FN = np.sum(y_sig[cut]) / integral_sig     # False negatives
            TPR[i] = TP / (TP + FN)                    # True positive rate
            
            # True negative
            TN = np.sum(y_bkg[cut]) / integral_bkg      # True negatives (background)
            FP = np.sum(y_bkg[~cut]) / integral_bkg     # False positives
            FPR[i] = FP / (FP + TN)                     # False positive rate            
            
        return FPR, TPR
    
    else:
        AssertionError("Signal and Background histograms have different bins and ranges")

### Fisher -discriminant

In [1]:
def fisher_disc(N, Hypo, *args):

    assert N == len(args) , f"*args has more arguments (len(args) = {len(args)}) as desired N = {len(N)}"
    
    marker_list = []
    for marker in args:
       
        # get events for each hpothesis
        h_0 = [x for i,x in enumerate(marker) if Hypo[i] == 0]
        h_1 = [x for i,x in enumerate(marker) if Hypo[i] == 1]
        
        marker_list = np.append(marker_list,{'0':h_0,'1':h_1})

    # get covariance matrix for null hypo.
    V_0 = np.array([cov_corr(x[0]['0'],x[1]['0'])[0] for x in  itertools.product(marker_list, marker_list)], 
                   dtype = np.float64)
    V_0 = np.reshape(V_0, (len(marker_list),len(marker_list)))
            
    # get covariance matrix for alternativ hypo.
    V_1 = np.array([cov_corr(x[0]['1'],x[1]['1'])[0] for x in  itertools.product(marker_list, marker_list)], 
                   dtype = np.float64)
    V_1 = np.reshape(V_1, (len(marker_list),len(marker_list)))


    mean_0 = [np.mean(x['0']) for x in marker_list]
    mean_1 = [np.mean(x['1']) for x in marker_list]
    


    W = V_0 + V_1
    alpha = np.dot(np.linalg.inv(W),(np.array(mean_0)-np.array(mean_1)))
    
    
    return W, alpha

### Some Nice Plotting Routines

In [3]:
def plot_hist_errorbar(H, plt_opt = True):
    # get binwidth
    dbin = np.abs(H[1][0]-H[1][1])
    
    # get midpoints of each bin 
    mid_bin = [np.abs(H[1][0]+H[1][1])/2 + i*dbin for i in range(len(H[1])-1)]
    
    # get error 
    error_bin = np.sqrt(H[0])
    
    if plt_opt == True:
        fig, ax = plt.subplots()
        # plot errobar
        ax.errorbar(mid_bin,H[0],yerr=error_bin, linestyle = " ", marker='o', mfc='coral',
         mec='coral', ms=3, mew=3)
        # plot step-func to show the underlying histogram
        ax.step(H[1][1::],H[0],alpha =0.6)
    
    # return (mid-point of bin, bin count, errors per bin) and min(bin count) 
    # to check for possible low count statistic
    return (mid_bin, H[0], error_bin), min(H[0]), ax



# CODE SNIPPETS

### Use of minuit

In [10]:
'''
chi2_pdf = Chi2Regression(func_pdf, test_x, count, np.sqrt(count))
# func_pdf is function, test_x is from Monte Carlo, count is the count in histogram, np.sqrt(count) is error of hist

minuit_pdf = Minuit(chi2_pdf, pedantic=False, A=2) 
# Stating wished function with guesses on paramters, here A=2. To fix parameters write fix_A = True

minuit_pdf.migrad();      
# Perform the actual fit


Ndof = len(test_x) - N_param           
# N_param is number of parameters in fit, here N_param = 1

Prob = scipy.stats.chi2.sf(minuit_pdf.fval, Ndof)
# Getting prbability of chi2 fitting
''';

### Example of Monte Carlo

In [11]:
'''
np.random.seed(42)
# In order to have same outpur always

ran_dist = np.random.rand(400)*2
# Creating 400 random variables in range [0;2]

BOX = C/2*ran_dist 
### WHY DID WE DIVIDE BY HALF?


ran_dist_func = C*(1-np.exp(-a*ran_dist))
# Creating numbers according to the wanted distribution

fig4, ax_4 = plt.subplots(nrows=1, ncols=2, figsize=(15, 5),sharex=True)

SAVE = []

while len(SAVE) < 500:
    ax_4[0].cla()
    SAVE = []
    ran_dist = np.random.rand(len(ran_dist)+100)*2
    # Creating enough numbers to satisfy problem
    
    for i in ran_dist:
        u = np.random.rand(1)
        if C*u <= C*(1-np.exp(-a*i)):
            ax_4[0].plot(i,C*u,marker ='o', color = 'red')
            SAVE.append(i)
        else:
            ax_4[0].plot(i,C*u,marker ='o', color = 'blue')

SAVE = np.array(SAVE[0:500])
# Making sure we have the stated number

HIST = ax_4[1].hist(SAVE,30);
ax_4[1].cla()
# We do not want to plot the histogram itself

ax_4[1].plot(np.sort(SAVE),[len(HIST[0])*C*(1-np.exp(-a*i)) for i in np.sort(SAVE)],marker=' ',linestyle='-')

count = HIST[0]

hist_std = np.sqrt(count)

test_x = np.linspace(HIST[1][0]+np.diff(HIST[1])[0]/2,HIST[1][-1],len(HIST[0]))
ax_4[1].errorbar(test_x,count,hist_std,linestyle =' ', marker ='o');
ax_4[1].set_ylabel('counts')
ax_4[1].set_xlabel('x')

fig4.savefig('MonteCarlo.png')

''';

### Slicing data in large histogram to smaller pieces

In [13]:
'''
data = np.array(x)
print("The number of entries in the file was: ", len(x))

fig_gamma, ax_gamma = plt.subplots()
H = ax_gamma.hist(data,1000);


window_low = 105
window_up = 170
# Choose which entries you want to slice to

ax_gamma.set_xlim(window_low, window_up)


x_data = [x for x in H[1] if x>window_low and x<window_up]
print(len(x_data))

y_data = [H[0][i] for i in range(len(H[1])) if H[1][i]>window_low and H[1][i]<window_up]
print(len(y_data))

# slice out data in window and make sure the code did it by printing

''';